In [1]:
# Soil Classification
# High-accuracy model with TTA, Cross-validation, and Ensembling

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm
import random

In [2]:
# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [3]:
# Define paths
TRAIN_DIR = "/kaggle/input/soil-classification/soil_classification-2025/train"
TEST_DIR = "/kaggle/input/soil-classification/soil_classification-2025/test"
LABELS_PATH = "/kaggle/input/soil-classification/soil_classification-2025/train_labels.csv"
TEST_IDS_PATH = "/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv"
SUBMISSION_PATH = "/kaggle/working/submission.csv"

In [4]:
# Load labels
labels_df = pd.read_csv(LABELS_PATH)
class_names = labels_df["soil_type"].unique().tolist()
class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}
idx_to_class = {idx: cls for cls, idx in class_to_idx.items()}
labels_df["label"] = labels_df["soil_type"].map(class_to_idx)

In [5]:
# Custom dataset class
class SoilDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["image_id"]
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.df.iloc[idx]["label"]
        return image, label

In [6]:
# Define transformations
image_size = 224
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
# Define model architecture
def create_model():
    model = models.efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))
    return model

In [8]:
# Training function
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0
    for images, labels in tqdm(dataloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

In [9]:
# Validation function
def validate(model, dataloader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    per_class_f1 = [f1_score(np.array(y_true)==i, np.array(y_pred)==i) for i in range(len(class_names))]
    return per_class_f1

In [10]:
# Cross-validation training
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models_list = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold, (train_idx, val_idx) in enumerate(kf.split(labels_df, labels_df.label)):
    print(f"\n===== Fold {fold+1} =====")
    train_df = labels_df.iloc[train_idx]
    val_df = labels_df.iloc[val_idx]

    train_ds = SoilDataset(train_df, TRAIN_DIR, transform=train_transform)
    val_ds = SoilDataset(val_df, TRAIN_DIR, transform=test_transform)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

    model = create_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(10):
        loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        f1s = validate(model, val_loader, device)
        print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Per-class F1: {np.round(f1s, 4)}, Min F1: {min(f1s):.4f}")

    models_list.append(model)


===== Fold 1 =====


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 110MB/s] 
100%|██████████| 31/31 [00:11<00:00,  2.75it/s]


Epoch 1, Loss: 0.5801, Per-class F1: [0.9423 0.8293 0.955  0.9213], Min F1: 0.8293


100%|██████████| 31/31 [00:06<00:00,  4.50it/s]


Epoch 2, Loss: 0.1890, Per-class F1: [0.9577 0.8642 0.9905 0.9011], Min F1: 0.8642


...

Epoch 8, Loss: 0.0777, Per-class F1: [1.     0.9873 1.     0.9892], Min F1: 0.9873


100%|██████████| 31/31 [00:07<00:00,  4.36it/s]


Epoch 9, Loss: 0.0546, Per-class F1: [1.     0.9873 0.9907 1.    ], Min F1: 0.9873


100%|██████████| 31/31 [00:07<00:00,  4.20it/s]


Epoch 10, Loss: 0.0497, Per-class F1: [0.9953 1.     1.     0.989 ], Min F1: 0.9890


In [11]:
# Inference with Test-Time Augmentation (TTA) and model ensembling

test_ids = pd.read_csv(TEST_IDS_PATH)  # Load the list of test image filenames

# Custom Dataset class for test images (used in earlier training, here mainly for consistency)
class TestDataset(Dataset):
    def __init__(self, df, img_dir, transform):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["image_id"]
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert("RGB")
        return self.transform(image)

# Define the TTA transform: random horizontal flips and small rotations to simulate different views
tta_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to generate TTA predictions using a given model and image
def tta_predict(model, image, n=5):
    model.eval()
    with torch.no_grad():
        preds = []
        for _ in range(n):
            img = tta_transform(image)
            img = img.unsqueeze(0).to(device)  
            output = model(img)
            preds.append(torch.softmax(output, dim=1).cpu().numpy())  # Get class probabilities
        mean_pred = np.mean(preds, axis=0)  # Average predictions over n augmentations
        return mean_pred.squeeze()  # Ensure shape is (num_classes,) not (1, num_classes)

# Run inference with ensembling and TTA
final_preds = []

for i in tqdm(range(len(test_ids)), desc="Generating Predictions"):
    img_id = test_ids.iloc[i]["image_id"]
    img_path = os.path.join(TEST_DIR, img_id)
    image = Image.open(img_path).convert("RGB")

    pred_sum = np.zeros(len(class_names))  # Accumulate predictions across models
    for model in models_list:
        pred = tta_predict(model, image)  # Get TTA prediction for this model
        pred_sum += pred  # Sum across ensemble models

    final_label = pred_sum.argmax()  # Choose class with highest average probability
    final_preds.append(idx_to_class[final_label])  # Convert index back to class name


Generating Predictions: 100%|██████████| 341/341 [01:53<00:00,  2.99it/s]


In [12]:
# Prepare submission DataFrame
submission_df = pd.DataFrame({
    "image_id": test_ids["image_id"],
    "soil_type": final_preds
})

In [13]:

# Save to CSV
submission_path = "submission.csv"
submission_df.to_csv(submission_path, index=False)
print(f" Submission file saved to: {submission_path}")

 Submission file saved to: submission.csv
